In [1]:
import os
import pandas as pd
# import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
input_df = pd.read_csv(
    '../data/b1-b9/MAB_b1-b9-22.02.17.csv',
    encoding="utf_16",
    sep=';'
)
input_df.head()

/home/ruslan/anaconda3/envs/streamlit/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,DateTime,IdRFID,IdLabel,unitLabel,eventDuration,sense1duration,sense1Events,senseRFIDrecords,reinforce1value,reinforce1Total,reinforce1Account,outFuncLabel,outLabel,SystemMsg,MsgValue1,MsgValue2,MsgValue3
0,#ID-Device,Sorter1.1,0,0,0.0,Sorter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,#ID-Device,Sorter1.2,0,0,0.0,Sorter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#ID-Device,Sorter1.3,0,0,0.0,Sorter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,#Sorter,Sorter1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#RfidCondition,CondMod1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def initial_cleaning(input_df):
    # sort the values since for some reason observations sometimes mixed in time
    input_df['DateTime'] = input_df['DateTime'].astype(float)
    input_df.sort_values(by='DateTime', inplace=True)
    input_df.reset_index(drop=True, inplace=True)

    # some datetime manipulations
    input_df['Timestamp'] = input_df['DateTime'].apply(lambda x: datetime.timestamp(from_ordinal(x)))
    input_df['DateTime'] = input_df['Timestamp'].apply(lambda x: datetime.fromtimestamp(x))

    return input_df


def from_ordinal(ordinal, _epoch=datetime(1899, 12, 30)):
    """Converts serial date-time to DateTime object.

    Parameters
    ----------
    ordinal : float or int
        Original serial date-time.
    _epoch : datetime
        Start of the count.
        NOTE: for some reason timestamp is shifted by 2 days
        backwards from 01-01-1900, that is why default value
        is set to 30-12-1899.
    """
    return _epoch + timedelta(days=ordinal)

In [4]:
input_df = input_df[~input_df['DateTime'].astype(str).apply(lambda x: x.startswith('#'))]

In [5]:
input_df = initial_cleaning(input_df)

In [6]:
ids = input_df['IdLabel'][~input_df['IdLabel'].isnull()].unique()
ids.sort()
ids_dict = dict(input_df[['IdLabel','IdRFID']].drop_duplicates().dropna().values)
ids_dict

{'b3': '041A716DCB',
 'b6': '041A7170BD',
 'b5': '041A71651C',
 'b2': '041A716801',
 'b7': '041A71679E',
 'b8': '041A717072',
 'b1': '041A71679D',
 'b9': '041A716804',
 'b4': '041A716CB6'}

In [7]:
# subj_data['MsgValue1'][cndtn]\
#     .reset_index(drop=True)\
#     .apply(lambda x: x.split('=')[1])

In [12]:
# input_df = initial_cleaning(input_df)

ids = input_df['IdLabel'][~input_df['IdLabel'].isnull()].unique()
ids.sort()
ids_dict = dict(input_df[['IdLabel', 'IdRFID']].drop_duplicates().dropna().values)

final_output = pd.DataFrame({})

for animal_id in tqdm(ids):
    # print(animal_id)
    indices_start = input_df[(input_df['IdLabel'] == animal_id) & (input_df['SystemMsg'] == 'start exp')].index
    indices_end = input_df[(input_df['IdLabel'] == animal_id) & (input_df['SystemMsg'] == 'end exp')].index

    for session_i in range(len(indices_start)):
        ind_start = indices_start[session_i]
        ind_end = indices_end[session_i]
        subj_data = input_df.iloc[ind_start:ind_end+1, :].reset_index(drop=True)

        cndtn = subj_data['SystemMsg'].apply(
            lambda x: x.startswith('start trial') if type(x) == str else False)
        total_trials = subj_data['SystemMsg'][cndtn].apply(lambda x: int(x.split(' ')[2])).max()
        total_outcomes = (subj_data['SystemMsg'] == 'Reward?').sum()

        wait_poke_ts = subj_data['Timestamp'][subj_data['SystemMsg'] == 'wait poke']\
            .reset_index(drop=True)

        trial_start_ts = subj_data['Timestamp'][cndtn]\
            .reset_index(drop=True)
        trial_start_ts.name = 'trialStart'

        trial_end_ts = subj_data['Timestamp'][subj_data['SystemMsg'] == 'start iti']\
            .reset_index(drop=True)
        if len(trial_start_ts) != len(trial_end_ts):
            trial_end_ts = trial_end_ts.append(
                pd.Series(
                    subj_data['Timestamp'][subj_data['SystemMsg'] == 'end exp']
                ),
                ignore_index=True)
        trial_end_ts.name = 'trialEnd'

        if len(wait_poke_ts) != len(trial_start_ts):
            wait_poke_ts = wait_poke_ts[:len(wait_poke_ts)-1]

        start_latency = trial_start_ts - wait_poke_ts
        start_latency.name = 'startLatency'

        trial_duration = trial_end_ts - trial_start_ts
        trial_duration.name = 'trialDuration'

        decision = subj_data['MsgValue1'][subj_data['SystemMsg'] == 'decision:']\
            .reset_index(drop=True)
        decision_n = decision.apply(lambda x: x.split(' ')[1])
        decision_n.name = 'decisionNumber'
        decision_pos = decision.apply(lambda x: x.split(' ')[2][2])
        decision_pos.name = 'decisionPosition'
        decision_img = decision.apply(lambda x: x.split(' ')[2][4])
        decision_img.name = 'decisionImage'

        decision_ts = subj_data['Timestamp'][subj_data['SystemMsg'] == 'decision:']\
            .reset_index(drop=True)
        decision_latency = decision_ts - trial_start_ts
        decision_latency.name = 'decisionLatency'

        reward = subj_data['MsgValue1'][subj_data['SystemMsg'] == 'Reward?']\
            .reset_index(drop=True)
        reward.name = 'reward'
        reward = reward == 'True'

        reward_ready_ts = subj_data['Timestamp'][subj_data['SystemMsg'] == 'reward ready']\
            .reset_index(drop=True)

        reward_collected_ts = subj_data['Timestamp'][subj_data['SystemMsg'] == 'reward collected']\
            .reset_index(drop=True)

        reward_latency = reward_collected_ts - reward_ready_ts
        reward_latency.index = reward[reward == True].index
        reward_latency.name = 'rewardLatency'

        p1 = subj_data['MsgValue1'][cndtn]\
            .reset_index(drop=True)\
            .apply(lambda x: x.split('=')[1])
        p1.name = 'P1'

        p2 = subj_data['MsgValue2'][cndtn]\
            .reset_index(drop=True)\
            .apply(lambda x: x.split('=')[1])
        p2.name = 'P2'

        p3 = subj_data['MsgValue3'][cndtn]\
            .reset_index(drop=True)\
            .apply(lambda x: x.split('=')[1])
        p3.name = 'P3'

        opt1 = subj_data['MsgValue1'][cndtn]\
            .reset_index(drop=True)\
            .apply(lambda x: x.split(' ')[2])
        opt1.name = 'Option1'

        opt2 = subj_data['MsgValue2'][cndtn]\
            .reset_index(drop=True)\
            .apply(lambda x: x.split(' ')[2])
        opt2.name = 'Option2'

        opt3 = subj_data['MsgValue3'][cndtn]\
            .reset_index(drop=True)\
            .apply(lambda x: x.split(' ')[2])
        opt3.name = 'Option3'

        session_out = pd.concat(
            [trial_start_ts, trial_end_ts, trial_duration, start_latency, opt1, opt2, opt3,
             p1, p2, p3, decision_n, decision_pos, decision_img, decision_latency, reward],
            axis=1)
        session_out = session_out.join(reward_latency)

        if total_trials != total_outcomes:
            session_out = session_out.iloc[:total_trials-1, :]

        session_out['trial'] = session_out.index + 1
        session_out['animalID'] = animal_id
        session_out['session'] = session_i + 1
        session_out['scenario'] = subj_data['MsgValue2'][0]

        final_output = final_output.append(session_out).reset_index(drop=True)

100%|██████████| 9/9 [00:01<00:00,  4.67it/s]


In [13]:
final_output = final_output[[
    'animalID', 'session', 'scenario','trial', 'trialStart', 'trialEnd', 'trialDuration',
    'startLatency', 'Option1', 'Option2', 'Option3', 'P1', 'P2', 'P3', 'decisionNumber', 'decisionPosition', 'decisionImage', 'decisionLatency', 'reward', 'rewardLatency'
]]

In [14]:
final_output

,animalID,session,scenario,trial,trialStart,trialEnd,trialDuration,startLatency,Option1,Option2,Option3,P1,P2,P3,decisionNumber,decisionPosition,decisionImage,decisionLatency,reward,rewardLatency
0,b1,1,Scenario 1,1,1.645121e+09,1.645121e+09,13.969004,12.247002,@P2_4,@P1_6,@P3_5,0.875,0.0625,0.0625,1,2,4,6.725997,True,2.237993
1,b1,1,Scenario 1,2,1.645121e+09,1.645121e+09,2.385002,4.266994,@P2_4,@P1_6,@P3_5,0.875,0.0625,0.0625,3,3,5,2.380000,False,NaN
2,b1,1,Scenario 1,3,1.645121e+09,1.645121e+09,10.181004,0.823996,@P2_4,@P1_6,@P3_5,0.875,0.0625,0.0625,1,2,4,3.149004,True,2.028006
3,b1,1,Scenario 1,4,1.645121e+09,1.645121e+09,7.044002,0.787994,@P2_4,@P1_6,@P3_5,0.875,0.0625,0.0625,2,1,6,7.039000,False,NaN
4,b1,1,Scenario 1,5,1.645121e+09,1.645121e+09,3.412005,3.425000,@P2_4,@P1_6,@P3_5,0.875,0.0625,0.0625,2,1,6,3.410001,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3200,b9,2,Scenario 1,96,1.645153e+09,1.645153e+09,17.358002,0.002005,@P2_6,@P3_5,@P1_4,0.875,0.0625,0.0625,1,2,6,10.454996,True,1.899003
3201,b9,2,Scenario 1,97,1.645153e+09,1.645153e+09,3.070993,2.236006,@P2_6,@P3_5,@P1_4,0.875,0.0625,0.0625,2,3,5,3.068998,False,NaN
3202,b9,2,Scenario 1,98,1.645153e+09,1.645153e+09,11.091004,6.118001,@P2_6,@P3_5,@P1_4,0.875,0.0625,0.0625,1,2,6,4.379003,True,1.709000
3203,b9,2,Scenario 1,99,1.645153e+09,1.645153e+09,9.316002,0.001996,@P2_6,@P3_5,@P1_4,0.875,0.0625,0.0625,1,2,6,2.635001,True,1.678994


In [12]:
pd.to_datetime(final_output['trialStart'], unit='s')

0     2021-12-09 18:17:34.713004032
1     2021-12-09 18:18:09.152001024
2     2021-12-09 18:18:20.000998144
3     2021-12-09 18:18:31.470001920
4     2021-12-09 18:18:52.580002048
                   ...             
879   2021-12-10 03:13:08.481003008
880   2021-12-10 03:13:27.434000128
881   2021-12-10 03:14:09.270996992
882   2021-12-10 03:14:29.319997184
883   2021-12-10 03:15:08.249996032
Name: trialStart, Length: 884, dtype: datetime64[ns]

In [16]:
final_output['trialStart']

0      1.638289e+09
1      1.638289e+09
2      1.638289e+09
3      1.638289e+09
4      1.638289e+09
           ...     
484    1.638304e+09
485    1.638304e+09
486    1.638304e+09
487    1.638304e+09
488    1.638304e+09
Name: trialStart, Length: 489, dtype: float64